In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import altair as alt

%load_ext autoreload
%autoreload 2

In [19]:
PATH = Path(r'D:\OneDrive - ANATEL\Monitoramento\VIC\PRD2020\Processamento')
NIVEL = PATH / '50_63_processado_sao_vicente.csv'
OC = PATH / '21_65_processado_rpo.csv'
PISO = -75

In [5]:
df = pd.read_csv(NIVEL, delimiter=';', decimal=',')
df.columns = ["Frequência", 'Máxima', 'Média']

In [13]:
nivel = pd.read_csv(NIVEL, delimiter=';', decimal=',', index_col=0)
#oc = pd.read_csv(OC, delimiter=';', decimal=',', index_col=0)

In [14]:
#oc.columns = 
nivel.columns = ['Máxima', 'Média']

In [15]:
nivel.index.rename('Frequência', inplace=True)
#oc.index = nivel.index

In [16]:
nivel = nivel.reset_index().melt('Frequência', var_name='Tipo', value_name='Nível')

In [17]:
nivel.head()

,Frequência,Tipo,Nível
0,170.000000,Máxima,-82.5
1,170.019539,Máxima,-82.5
2,170.039078,Máxima,-82.5
3,170.058617,Máxima,-82.5
4,170.078155,Máxima,-83.0


In [28]:
oc = oc.reset_index().melt('Frequência', var_name='Tipo', value_name='Ocupação')

In [29]:
oc.head()

,Frequência,Tipo,Ocupação
0,108.000000,Máxima,1.0
1,108.001953,Máxima,0.0
2,108.003907,Máxima,0.5
3,108.005860,Máxima,0.5
4,108.007813,Máxima,0.0


In [96]:
df = pd.merge(nivel, oc, on=['Frequência', "Tipo"]).sort_values('Frequência').reset_index(drop=True)

In [18]:
df = nivel

In [111]:
df.to_excel(PATH / 'dados.xlsx')

In [114]:
df[(df.Nível >= PISO) & (df.Ocupação >= 1)]

,Frequência,Tipo,Nível,Ocupação
0,108.000000,Máxima,-57.0,1.0
10232,117.992861,Máxima,-56.0,1.0
10234,117.994814,Máxima,-43.5,2.5
10236,117.996767,Máxima,-32.0,5.0
10238,117.998720,Máxima,-28.5,5.0
10240,118.000674,Máxima,-32.0,4.5
10242,118.002627,Máxima,-42.5,2.0
11825,119.547653,Máxima,-51.0,3.0
11827,119.549606,Máxima,-50.5,1.5
11829,119.551559,Máxima,-56.0,1.5


In [23]:
df = df[df.Nível > PISO]

In [24]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=['Frequência'], empty='none')

In [25]:
# The basic line
line_1 = alt.Chart(df).mark_line(interpolate='basis').encode(
    x='Frequência:Q',
    y='Nível:Q',
    color='Tipo:N'
)

In [22]:
line_1

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)

In [81]:
# The basic line
line_2 = alt.Chart(df).mark_line(interpolate='basis').encode(
    x='Frequência:Q',
    y='Ocupação:Q',
    color='Tipo:N'
)

In [82]:
line_2

alt.Chart(...)

In [83]:
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df).mark_point().encode(
    x='Frequência:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

In [84]:
# Draw points on the line, and highlight based on selection
points_1 = line_1.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

In [85]:
# Draw points on the line, and highlight based on selection
points_2 = line_2.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

In [86]:
# Draw text labels near the points, and highlight based on selection
text_1 = line_1.mark_text(align='left', dx=5, dy=-5).encode(
                     text=alt.condition(nearest, 'Nível:Q', alt.value(' ')))

In [87]:
# Draw text labels near the points, and highlight based on selection
text_2 = line_2.mark_text(align='left', dx=5, dy=-15).encode(
                     text=alt.condition(nearest, 'Ocupação:Q', alt.value(' ')))

In [88]:
# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='Frequência:Q',
).transform_filter(
    nearest
)

In [89]:
# Draw text labels near the points, and highlight based on selection
text_3 = rules.mark_text(align='left', dx=70, dy=15).encode(
                     text=alt.condition(nearest, 'Frequência:Q', alt.value(' ')))

In [90]:
# Put the five layers into a chart and bind the data
chart = alt.layer(line_1, line_2, selectors, points_1, points_2, rules, text_1, text_2, text_3).properties(
    width=1920, height=1080)

In [91]:
chart.save(f'{PATH}/108-137.html', 'html')